In [51]:
import pandas as pd
import time
import numpy as np
import json
import os
from collections import defaultdict
from heavy_tail_observations import BothSideWeibullNoise, BothSideParetoNoise, BothSideFrechetNoise
from heavy_tail_lin_bandit import MENU, TOFU, SupHvyLinBandit

from matplotlib import pyplot as plt
%matplotlib inline
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
data = pd.read_csv("/home/rai/다운로드/train_triplets.txt", sep="\t", names=['user', 'song', 'number of play'])
data = data[:100000]
data.head()

,user,song,number of play
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1


In [11]:
user = np.unique(data['user'].to_numpy())
len(user)

2042

In [12]:
song = np.unique(data['song'].to_numpy())
len(song)

44458

In [4]:
tracks = pd.read_csv("/home/rai/다운로드/unique_tracks.txt", sep="<SEP>", names=['track', 'song', 'artist', 'title'])
tr = tracks[['track','song']]
tr

/tmp/ipykernel_26587/4175115434.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  tracks = pd.read_csv("/home/rai/다운로드/unique_tracks.txt", sep="<SEP>", names=['track', 'song', 'artist', 'title'])


,track,song
0,TRMMMYQ128F932D901,SOQMMHC12AB0180CB8
1,TRMMMKD128F425225D,SOVFVAK12A8C1350D9
2,TRMMMRX128F93187D9,SOGTUKN12AB017F4F1
3,TRMMMCH128F425532C,SOBNYVR12A8C13558C
4,TRMMMWA128F426B589,SOHSBXH12A8C13B0DF
...,...,...
999995,TRYYYUS12903CD2DF0,SOTXAME12AB018F136
999996,TRYYYJO128F426DA37,SOXQYIQ12A8C137FBB
999997,TRYYYMG128F4260ECA,SOHODZI12A8C137BB3
999998,TRYYYDJ128F9310A21,SOLXGOR12A81C21EB7


In [5]:
tag_list = pd.read_csv('/home/rai/다운로드/lastfm_unique_tags.txt',sep='\t',names=['genre','counts'])
tag_list

,genre,counts
0,rock,101071
1,pop,69159
2,alternative,55777
3,indie,48175
4,electronic,46270
...,...,...
522361,´rock,1
522362,Örebro,1
522363,äh,1
522364,ångest,1


In [6]:
tag_freq = tag_list[tag_list['counts']>10000]
tag_freq = list(tag_freq['genre'])
tag_freq

['rock',
 'pop',
 'alternative',
 'indie',
 'electronic',
 'female vocalists',
 'favorites',
 'Love',
 'dance',
 '00s',
 'alternative rock',
 'jazz',
 'beautiful',
 'singer-songwriter',
 'metal',
 'chillout',
 'male vocalists',
 'Awesome',
 'classic rock',
 'soul',
 'indie rock',
 'Mellow',
 'electronica',
 '80s',
 'folk',
 'british',
 '90s',
 'chill',
 'american',
 'instrumental',
 'punk',
 'oldies',
 'seen live',
 'blues',
 'hard rock',
 'cool',
 'Favorite',
 'ambient',
 'acoustic',
 'experimental',
 'Favourites',
 'female vocalist',
 'guitar',
 'Hip-Hop',
 '70s',
 'party',
 'country',
 'easy listening',
 'sexy',
 'catchy',
 'funk',
 'favourite',
 'electro',
 'heavy metal',
 'Progressive rock',
 '60s',
 'fun',
 'rnb',
 'indie pop',
 'Soundtrack',
 'loved',
 'sad',
 'House',
 'favorite songs',
 'happy',
 'punk rock',
 'piano',
 'psychedelic',
 'hip hop',
 'male vocalist',
 'classic',
 'pop rock',
 'downtempo',
 'trance',
 'melancholy',
 'female',
 'amazing',
 'hardcore',
 'rap',
 'lou

In [7]:
user1 = data.loc[data['user']=='b80344d063b5ccb3212f76538f3d9e43d87dca9e']
user1

,user,song,number of play
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1
...,...,...,...
99,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOYHEPA12A8C13097F,8
100,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOYYWMD12A58A7BCC9,1
101,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOZGCUB12A8C133997,1
102,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOZOBWN12A8C130999,1


In [8]:
user1 = pd.merge(user1, tr)
user1 = user1.drop(['song'],axis=1)
user1

,user,number of play,track
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,TRIQAUQ128F42435AD
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,TRIRLYL128F42539D1
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,2,TRMHBXZ128F4238406
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,TRYQMNI128F147C1C7
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,TRAHZNE128F9341B86
...,...,...,...
100,b80344d063b5ccb3212f76538f3d9e43d87dca9e,8,TRVPAII128F42435A8
101,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,TRLUAWR128F4230F26
102,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,TRXEMPO128F424602E
103,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,TRQLOHS128F42435AE


In [9]:
with open('/home/rai/다운로드/lastfm_train/A/A/A/TRAAAAK128F9318786.json') as f:
    dt = json.load(f)
dt

{'artist': 'Adelitas Way',
 'timestamp': '2011-08-15 09:59:32.436152',
 'similars': [['TRZNRZF128F9318787', 1],
  ['TRRJFIC128F931879D', 0.966398],
  ['TRLYTDK128F93187BB', 0.158876],
  ['TRQYGQT128F9318777', 0.157267],
  ['TRJYOJG128F93187B4', 0.157267],
  ['TROLOPS128F426E353', 0.11712],
  ['TRQNBAM128F9336E65', 0.107759],
  ['TRILDOA128F426E75F', 0.106884],
  ['TRVFCII128F92F6E7E', 0.0915493],
  ['TRYODCQ12903C9595C', 0.0912093],
  ['TRKTPWS128F427EDCC', 0.0900865],
  ['TRUWAVU128F9336E64', 0.0892342],
  ['TRCXFJG128F427EDC9', 0.0869064],
  ['TRTIYNH128F425249F', 0.0847005],
  ['TRGYGWR128F9338FCB', 0.0824838],
  ['TRLTCTY128F14A65FA', 0.0824838],
  ['TRDSEGT12903D02A53', 0.0824838],
  ['TRZYCVZ12903D02A51', 0.0815291],
  ['TRTPLGZ128EF3405DC', 0.0815291],
  ['TRJQMZC128F9338FC5', 0.0815291],
  ['TRJEFFA128F932106F', 0.0815291],
  ['TRVZNPG128F14A65EF', 0.0815291],
  ['TRKQAWL128F92EB536', 0.0801389],
  ['TRANFIJ128F9366F6F', 0.0800814],
  ['TRQMWBS128F9366F7C', 0.0793406],
  ['TRFX

In [10]:
df = pd.DataFrame(columns=['track', 'tags'])
tags = []
for t in user1['track']:
    path = '/home/rai/다운로드/lastfm_train/' + t[2] + '/' + t[3] + '/' + t[4]
    if t + '.json' in os.listdir(path):
        with open(path + '/' + t + '.json') as f:
            dt = json.load(f)
        df2 = pd.DataFrame({'track' : dt['track_id'], 'tags' : dt['tags']})
        df = df.append(df2, ignore_index=True)
df

/tmp/ipykernel_26587/3311443578.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2, ignore_index=True)


,track,tags
0,TRIQAUQ128F42435AD,"[instrumental, 100]"
1,TRIQAUQ128F42435AD,"[acoustic, 90]"
2,TRIQAUQ128F42435AD,"[guitar, 50]"
3,TRIQAUQ128F42435AD,"[Mellow, 40]"
4,TRIQAUQ128F42435AD,"[chill, 40]"
...,...,...
4624,TRQLOHS128F42435AE,"[private stashshsh, 1]"
4625,TRQLOHS128F42435AE,"[acoustirock, 1]"
4626,TRQLOHS128F42435AE,"[folksy indie rock, 1]"
4627,TRQLOHS128F42435AE,"[My Songs, 1]"


In [11]:
unique_track = set(df['track'])
unique_track

{'TRAAHSY128F147BB5C',
 'TRADVZX128F426BF79',
 'TRAHZNE128F9341B86',
 'TRAUCNU128F42671EB',
 'TRBAYHO128F92F0778',
 'TRCUEPI128F4278E10',
 'TRCUSGT128F147BD30',
 'TRDCZUX128F4263A74',
 'TRDLMWP128F426BF6C',
 'TRDMBIJ128F4290431',
 'TRDSKCB128F4256A11',
 'TRDUAJB128F147C1C6',
 'TRDWQHL128F147BD31',
 'TREJOUZ128F4278FCE',
 'TRFUPBA128F934F7E1',
 'TRGOSGL128F1471533',
 'TRHDDQG12903CB53EE',
 'TRHNCIR128F42334A5',
 'TRHUEGH128F4224BE0',
 'TRIBEAC128F147C1C8',
 'TRIDGUW128F9359DBC',
 'TRIJNYC128F146D22A',
 'TRIQAUQ128F42435AD',
 'TRIRLYL128F42539D1',
 'TRISTCY12903CD6D50',
 'TRJPULA128F147BD32',
 'TRJXMQA12903CD6D5A',
 'TRKAJKO128F1459199',
 'TRKEXHB128F147C1C4',
 'TRKKHXU128F92E87A9',
 'TRKRHYM128F42934A9',
 'TRKTRKM128F4280BB6',
 'TRKUSXZ128F4277AF3',
 'TRLQPQJ128F42AA94F',
 'TRLUAWR128F4230F26',
 'TRLXSNR128F429361D',
 'TRMHBXZ128F4238406',
 'TRMSZXT128F92F7816',
 'TRMXEXM12903CD6D57',
 'TRNRDZV128F4277AC9',
 'TRPDNGZ128F4236B10',
 'TRPLAXZ128F4292406',
 'TRPSCQO128F148978D',
 'TRPTWGR12

In [12]:
tag_set = set([])
for k in df['tags']:
    if k[0] in tag_freq:
        tag_set.add(k[0])

tag_set

{'00s',
 '60s',
 '70s',
 '80s',
 '90s',
 'Awesome',
 'Favorite',
 'Favourites',
 'Hip-Hop',
 'Love',
 'Mellow',
 'Progressive rock',
 'Soundtrack',
 'acoustic',
 'alternative',
 'alternative rock',
 'amazing',
 'ambient',
 'american',
 'beautiful',
 'blues',
 'british',
 'catchy',
 'chill',
 'chillout',
 'classic',
 'classic rock',
 'cool',
 'country',
 'cover',
 'dance',
 'downtempo',
 'easy listening',
 'electro',
 'electronic',
 'electronica',
 'experimental',
 'favorite songs',
 'favorites',
 'favourite',
 'female',
 'female vocalist',
 'female vocalists',
 'folk',
 'fun',
 'funk',
 'guitar',
 'happy',
 'hard rock',
 'heard on Pandora',
 'hip hop',
 'indie',
 'indie pop',
 'indie rock',
 'instrumental',
 'jazz',
 'lounge',
 'loved',
 'male vocalist',
 'male vocalists',
 'melancholy',
 'metal',
 'new wave',
 'oldies',
 'party',
 'piano',
 'pop',
 'pop rock',
 'psychedelic',
 'punk',
 'punk rock',
 'rap',
 'reggae',
 'relax',
 'rnb',
 'rock',
 'sad',
 'seen live',
 'sexy',
 'singer-s

In [83]:
df2 =  pd.DataFrame(columns = ['track', 'number of play'] + list(tag_set))
for t in unique_track:
    df_t = df[df['track'] == t]
    g = []
    for ta in list(df_t['tags']):
        if ta[0] in tag_set:
            g.append(ta)
    dic = {'track' : t, 'number of play' : list(user1[user1['track']==t]['number of play'])[0]}
    for i in g:
        dic[i[0]] = i[1]
        
    df3 = pd.DataFrame(dic, columns = ['track', 'number of play'] + list(tag_set), index=[0])
    df2 = df2.append(df3, ignore_index=True)
df2 = df2.fillna(0)
df2

,track,number of play,country,heard on Pandora,acoustic,Favourites,pop rock,lounge,trance,folk,...,blues,downtempo,chill,instrumental,punk rock,Favorite,female vocalist,experimental,hip hop,indie rock
0,TRGOSGL128F1471533,1,0,0,0,0,0,20,0,0,...,0,40,20,0,0,0,0,0,0,0
1,TRVMUOP128F42671F4,1,0,0,4,0,0,0,0,4,...,0,0,0,4,0,0,0,0,0,0
2,TRMXEXM12903CD6D57,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,TRVIXHO128F92C68A6,1,0,0,60,0,0,0,0,80,...,0,0,0,0,0,20,0,0,0,0
4,TRZTHKS128F92EC098,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,TRSPGGJ128F428081A,1,0,0,0,2,0,0,0,0,...,0,0,4,0,0,21,0,0,0,7
78,TRDLMWP128F426BF6C,1,0,0,11,0,0,0,0,100,...,0,0,7,0,0,0,0,0,0,19
79,TRPUGUW128F426BF6F,1,0,0,13,0,0,0,0,93,...,0,0,3,0,0,0,0,0,0,17
80,TRJPULA128F147BD32,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [85]:
df2 = df2[:81]
df3 = df2.drop(['number of play', 'track'],axis=1)
df3 = df3.astype(float)
df3 = df3 / 100
df3[:81]

,country,heard on Pandora,acoustic,Favourites,pop rock,lounge,trance,folk,chillout,indie pop,...,blues,downtempo,chill,instrumental,punk rock,Favorite,female vocalist,experimental,hip hop,indie rock
0,0.0,0.0,0.00,0.00,0.0,0.2,0.0,0.00,1.0,0.00,...,0.0,0.4,0.20,0.00,0.0,0.00,0.0,0.0,0.0,0.00
1,0.0,0.0,0.04,0.00,0.0,0.0,0.0,0.04,0.0,0.00,...,0.0,0.0,0.00,0.04,0.0,0.00,0.0,0.0,0.0,0.00
2,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.00,...,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00
3,0.0,0.0,0.60,0.00,0.0,0.0,0.0,0.80,0.0,0.20,...,0.0,0.0,0.00,0.00,0.0,0.20,0.0,0.0,0.0,0.00
4,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.00,...,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.00,...,0.0,0.0,0.00,0.12,0.0,0.00,0.0,0.0,0.0,0.00
77,0.0,0.0,0.00,0.02,0.0,0.0,0.0,0.00,0.0,0.00,...,0.0,0.0,0.04,0.00,0.0,0.21,0.0,0.0,0.0,0.07
78,0.0,0.0,0.11,0.00,0.0,0.0,0.0,1.00,0.0,0.09,...,0.0,0.0,0.07,0.00,0.0,0.00,0.0,0.0,0.0,0.19
79,0.0,0.0,0.13,0.00,0.0,0.0,0.0,0.93,0.0,0.22,...,0.0,0.0,0.03,0.00,0.0,0.00,0.0,0.0,0.0,0.17


In [86]:
y = df2['number of play'].to_numpy()
X = df3.to_numpy()
dim = X.shape[1]
total_samples = len(y)

T = 27
K = int(total_samples/T)

p=1.5
S = 1.

D = np.reshape(X, (T,K,dim))
R = np.reshape(y, (T,K))

In [102]:
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       5, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 6, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1,
       1, 5, 1, 1, 1, 8, 1, 1, 1, 2, 1, 1, 1, 1, 1])

In [87]:
get_mean = lambda x, y: R[y]
get_observation = lambda x, y, z: R[y][z]

In [100]:
# error_list, theta_star = MENU(D, get_mean, get_observation, S=S, lamb=1., delta=0.01, p=p, c=1.)
error_list, theta_star = TOFU(D, get_mean, get_observation, S=S, lamb=1., delta=0.01, p=p, b=1.)
# error_list, pro_estimator = SupHvyLinBandit(D, get_mean, get_observation, method="proof", S=S, lamb=1., delta=0.001, nu = 1e-4, p=p)
# error_list, btc_estimator = SupHvyLinBandit(D, get_mean, get_observation, method="btc", S=S, lamb=1., delta=0.001, nu = 1., p=p)
# error_list, bmm_estimator = SupHvyLinBandit(D, get_mean, get_observation, method="bmm", S=S, lamb=1., delta=0.001, nu = 1., p=p)

In [101]:
error_list

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.5       , 0.44444444, 0.5       ,
       0.45454545, 0.75      , 0.69230769, 0.64285714, 0.6       ,
       0.5625    , 0.82352941, 0.77777778, 0.73684211, 0.75      ,
       0.71428571, 0.72727273, 0.86956522, 1.125     , 1.08      ,
       1.07692308, 1.03703704])